In [1]:
!pip install --upgrade gtfs-realtime-bindings
!pip install mplleaflet

In [20]:
from google.transit import gtfs_realtime_pb2
import urllib.request
import mplleaflet as mp
import pandas as pd
import matplotlib.pyplot as plt
import time
import pickle

In [4]:
dir = '../data/static_data'
route = pd.read_csv(dir + 'routes.txt',sep=",")
route

,route_short_name,route_long_name,route_type,route_id,agency_id
0,NaN,108DOWN,3,0,DIMTS
1,NaN,108STLDOWN,3,1,DIMTS
2,NaN,108STLUP,3,2,DIMTS
3,NaN,108UP,3,3,DIMTS
4,NaN,185DOWN,3,4,DIMTS
...,...,...,...,...,...
1540,NaN,624ACLDOWN_DTC,3,1540,DTC
1541,NaN,408CLDOWN_DTC,3,1541,DTC
1542,NaN,114UP_DTC,3,1543,DTC
1543,NaN,182ACLSTLU_DTC,3,1545,DTC


In [5]:
trip = pd.read_csv(dir + 'trips.txt',sep=',')
trip

,route_id,service_id,trip_id,shape_id
0,0,1,0_6_0,0.0
1,0,1,0_6_10,0.0
2,0,1,0_6_20,0.0
3,0,1,0_6_30,0.0
4,0,1,0_6_40,0.0
...,...,...,...,...
75752,1544,1,1544_15_25,NaN
75753,1544,1,1544_8_31,NaN
75754,1544,1,1544_17_7,NaN
75755,1544,1,1544_13_55,NaN


In [6]:
df=route.merge(trip,how='inner',on='route_id')[['route_long_name','route_id','shape_id']]
df

,route_long_name,route_id,shape_id
0,108DOWN,0,0.0
1,108DOWN,0,0.0
2,108DOWN,0,0.0
3,108DOWN,0,0.0
4,108DOWN,0,0.0
...,...,...,...
75752,114DOWN_DTC,1544,NaN
75753,114DOWN_DTC,1544,NaN
75754,114DOWN_DTC,1544,NaN
75755,114DOWN_DTC,1544,NaN


In [7]:
route['agency_id'].value_counts()

DIMTS    1210
DTC       335
Name: agency_id, dtype: int64

In [8]:
shapes = pd.read_csv(dir + 'shapes.txt', sep=",")
shapes

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,0,28.625198,77.110808,0
1,0,28.625220,77.110815,1
2,0,28.625280,77.110831,2
3,0,28.625483,77.110835,3
4,0,28.625705,77.110839,4
...,...,...,...,...
242239,533,28.618988,77.031115,708
242240,533,28.619000,77.031478,709
242241,533,28.619005,77.031547,710
242242,533,28.618953,77.031561,711


In [9]:
route_shape_map = df.merge(shapes,how='inner',on='shape_id')

In [10]:
route_shape_map.drop_duplicates(inplace=True)
route_shape_map

,route_long_name,route_id,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,108DOWN,0,0.0,28.625198,77.110808,0
1,108DOWN,0,0.0,28.625220,77.110815,1
2,108DOWN,0,0.0,28.625280,77.110831,2
3,108DOWN,0,0.0,28.625483,77.110835,3
4,108DOWN,0,0.0,28.625705,77.110839,4
...,...,...,...,...,...,...
16460992,751UP,533,533.0,28.618988,77.031115,708
16460993,751UP,533,533.0,28.619000,77.031478,709
16460994,751UP,533,533.0,28.619005,77.031547,710
16460995,751UP,533,533.0,28.618953,77.031561,711


# Graph creation

In [11]:
x = route_shape_map.groupby(['shape_pt_lat','shape_pt_lon']).size()
x

shape_pt_lat  shape_pt_lon
28.490695     77.304077        1
28.490785     77.303741       10
28.490846     77.304046        6
28.491244     77.303870       11
28.491317     77.304038        2
                              ..
28.858176     77.093328        1
              77.093332        1
              77.093337        1
              77.093342        1
              77.093347        1
Length: 45848, dtype: int64

In [12]:
# graph = {}

In [13]:
# for index,row in route_shape_map.iterrows():
#   x = row.to_dict()
#   gps = (x['shape_pt_lat'],x['shape_pt_lon'])
#   if gps in graph:
#     temp = x['route_id']
#     if temp not in graph[gps]['route']:
#       graph[gps]['route'].append(temp)
  
#   else:
#     graph[gps]={}
#     graph[gps]['route']=[]
#     graph[gps]['route'].append(x['route_id'])

# graph

In [14]:
# %%time
# for key in graph:
#   graph[key]['neg'] = []
#   h=route_shape_map[(route_shape_map['shape_pt_lat']==key[0]) & (route_shape_map['shape_pt_lon']==key[1])]
#   for index,row in h.iterrows():
#     x = row.to_dict()
#     temp = route_shape_map[(route_shape_map['route_id']==x['route_id']) & (route_shape_map['shape_pt_sequence']==x['shape_pt_sequence']+1)]
#     records = temp[['shape_pt_lat','shape_pt_lon']].to_records(index=False)
#     result = list(set(list(records)))
#     graph[key]['neg'].extend(result)

In [15]:
# %%time
# for i, key in enumerate(graph):
#     if i%1000 == 0:
#         print(f'{i}/{len(graph)}')
#     graph[key]['neg'] = []
#     h = route_shape_map[(route_shape_map['shape_pt_lat']==key[0]) & (route_shape_map['shape_pt_lon']==key[1])].values.tolist()
#     h2 = route_shape_map[route_shape_map['route_id'].isin(graph[key]['route'])]
#     for row in h:
#         temp = h2[(h2['route_id']==row[1]) & (h2['shape_pt_sequence']==row[-1]+1)]
#         records = temp[['shape_pt_lat','shape_pt_lon']].to_records(index=False)
#         result = list(records)
#         graph[key]['neg'].extend(result)
# for key in graph:
#   graph[key]['neg'] = list(set([tuple(i) for i in graph[key]['neg']]))
# graph

In [16]:
# with open('graph.pkl','wb') as f:
#   pickle.dump(graph,f)

In [21]:
with open('graph.pkl','rb') as f:
  graph = pickle.load(f)

In [24]:
for key in graph:
    print(graph[key])
    break

{'route': [0, 466, 532], 'neg': [(28.625220000000002, 77.110815)]}


##Trie


In [ ]:
class TrieNode:
    Node_size = len(graph) 
    def __init__(self, loc=None, startRoutes=None, endRoutes=None, neighbours=None):
        self.loc = loc
        self.neighbours = neighbours
        self.startRoutes = startRoutes
        self.endRoutes = endRoutes

class Trie:
    def __init__(self):
        self.root = TrieNode()
    
    def charToIndex(self, ch):
        return ord(ch) - ord('a')

    def add(self, ):
        pCrawl = self.root
        length = len(word)
        for level in range(length):
            index = self.charToIndex(word[level])
            if not pCrawl.children[index]:
                pCrawl.children[index] = TrieNode()
            pCrawl = pCrawl.children[index]
        pCrawl.endOfWord = True
    
    def search(self, word):
        pCrawl = self.root
        length = len(word)
        for level in range(length):
            index = self.charToIndex(word[level])
            if not pCrawl.children[index]:
                return False
            pCrawl = pCrawl.children[index]
        return pCrawl.endOfWord


trie = Trie()
words = ["example", "extempore", "exterminate"]
for word in words:
    trie.add(word)
print(trie.search("extempore"))
print(trie.search("exterm"))

#Check


In [ ]:
def put(feed,lat,lon):
  # print(feed.entity)
  for entity in feed.entity:
    if entity.id=="DL1PC6641":
      lat.append(entity.vehicle.position.latitude)
      lon.append(entity.vehicle.position.longitude)
      break

In [ ]:
feed = gtfs_realtime_pb2.FeedMessage()
response = urllib.request.urlopen('https://opendata.iiitd.edu.in/api/realtime/VehiclePositions.pb?key=5kOKha1VJqxWvkkIp1J71Yrb5mc3VsDS')
feed.ParseFromString(response.read())
print(feed.entity)

In [ ]:
feed = gtfs_realtime_pb2.FeedMessage()
lat=[]
lon=[]
for i in range(100):
  response = urllib.request.urlopen('https://opendata.iiitd.edu.in/api/realtime/VehiclePositions.pb?key=5kOKha1VJqxWvkkIp1J71Yrb5mc3VsDS')
  feed.ParseFromString(response.read())
  put(feed,lat,lon)
  time.sleep(10)

In [ ]:
lat

In [ ]:
fig =plt.figure()
plt.plot(lon,lat,linewidth=3)
mp.display(fig=fig)